In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

C:\Users\Amit\Anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [7]:
df=pd.read_csv('Original_Features.csv')
df=df.drop(['rolloff','zero_crossing_rate'],axis=1)
df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,chroma_stft,spectral_centroid,spectral_bandwidth,label
0,-304.234070,94.326431,-7.126974,33.175732,7.065147,-0.331963,-10.314158,2.147586,-2.013190,5.324332,-0.093334,2.562227,1.275930,0.429254,2478.257487,2170.345949,1.0
1,-309.019653,79.845779,-5.289888,21.285072,-12.034595,-0.241961,-9.138373,0.444917,-13.440741,2.820725,-14.505177,12.173492,-7.108042,0.442904,2804.672224,2265.926588,1.0
2,-379.454376,115.356438,-11.129870,49.783417,-6.287680,4.333356,-10.013806,-10.836457,-21.100973,7.533222,-7.160056,-7.462425,-5.815040,0.404891,1911.933221,1695.401009,1.0
3,-276.964905,75.217216,-3.998095,13.369978,4.086710,-6.022980,-14.943695,-17.998863,-26.322065,-2.699209,-13.864196,-7.851540,-11.660230,0.411506,2820.892019,2428.570850,0.0
4,-284.207062,81.567101,26.056236,38.064480,14.861828,6.161382,-27.253889,-6.253015,-4.060058,-1.371932,-8.508703,1.715142,5.017691,0.325075,2367.901852,2403.353350,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17225,-281.850769,63.371513,4.073536,25.501684,4.071187,1.362235,-34.401535,-14.963532,-8.662941,5.242755,-18.567642,2.362002,-8.896091,0.359131,2925.769158,2453.494659,1.0
17226,-352.283020,147.483002,31.588148,44.751774,23.768219,1.736998,-2.201241,-4.397760,-4.463058,-1.630170,-2.593652,2.654396,-5.202733,0.443625,1103.973553,1732.738344,0.0
17227,-283.886566,70.052811,-15.057649,28.327362,1.277114,-6.033658,-5.651168,-3.200036,-13.372633,2.119324,-4.196598,0.092084,-9.053541,0.476382,2886.058571,2332.577729,0.0
17228,-329.493103,152.794022,-23.235121,38.832031,-9.015368,4.381469,-4.818068,-5.893717,-12.196824,-0.452028,-13.086767,-6.217406,1.336209,0.364045,1575.800627,1485.843688,0.0


In [ ]:
# All Features
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10, random_state=42)
tpot = TPOTClassifier(generations=7, population_size=70, verbosity=2, random_state=42, n_jobs=-1)
tpot.fit(X_train, y_train)
#tpot.export('tpot_exported_pipeline.py')
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')


Generation 1 - Current best internal CV score: 0.8231791870137771

Generation 2 - Current best internal CV score: 0.8231791870137771

Generation 3 - Current best internal CV score: 0.8447150323381052

Generation 4 - Current best internal CV score: 0.8605145778768563

Generation 5 - Current best internal CV score: 0.8605145778768563

Generation 6 - Current best internal CV score: 0.8605145778768563

Generation 7 - Current best internal CV score: 0.8605145778768563

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.7500000000000001, min_samples_leaf=2, min_samples_split=6, n_estimators=100)
0.8845037724898432


In [22]:
# %load "tpot_pipeline.py"
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = df
features = tpot_data.iloc[:,:-1]
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, df.iloc[:,-1], random_state=42)

# Average CV score on the training set was: 0.8605145778768563
exported_pipeline = ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.7500000000000001, min_samples_leaf=2, min_samples_split=6, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(training_features, training_target)
y_pred = exported_pipeline.predict(testing_features)

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 
from pycm import ConfusionMatrix
# Function to calculate accuracy 
def cal_accuracy(y_test, y_pred): 
    print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 
    print("Report : ", classification_report(y_test, y_pred)) 
    print(ConfusionMatrix(actual_vector=list(y_test),predict_vector=list(y_pred)))

cal_accuracy(testing_target, y_pred)

Accuracy :  86.76880222841226
Report :                precision    recall  f1-score   support

         0.0       0.87      0.90      0.88      2414
         1.0       0.86      0.83      0.85      1894

    accuracy                           0.87      4308
   macro avg       0.87      0.86      0.87      4308
weighted avg       0.87      0.87      0.87      4308

Predict    0.0        1.0        
Actual
0.0        2169       245        

1.0        325        1569       





Overall Statistics : 

95% CI                                                            (0.85757,0.87781)
ACC Macro                                                         0.86769
ARI                                                               0.54051
AUNP                                                              0.86346
AUNU                                                              0.86346
Bennett S                                                         0.73538
CBA                                      